In [1]:
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import itertools
import glob
from openpyxl import load_workbook
Auftrag = './absys_files/A-2017-0059_919A.XLSX'

In [2]:
def findZählernummer(ZNr, df):
    if isinstance(int(ZNr), int):
        assert len(str(ZNr)) == 8
        return df[(df['ZählernummerAlt'] == ZNr)]

In [3]:
def protoCleanupHeizung(df):
    df2append = df.dropna()
    df2append = df2append[(df2append['Baulänge'] != 'Baulänge') &
                          (df2append['Bauhöhe'] != 'Bauhöhe')&
                          (df2append['Bautiefe'] != 'Bautiefe') &
                          (df2append['ZählernummerAlt'] != 'Altgerät') &
                          (df2append['ZählernummerNeu'] != 'Neugerät')]
    df2append = df2append[(df2append['Baulänge'] != 'D') &
                          (df2append['Bauhöhe'] != 'E') &
                          (df2append['Bautiefe'] != 'F') &
                          (df2append['ZählernummerAlt'] != 'Nr.') &
                          (df2append['ZählernummerNeu'] != 'Nr.')]
    return df2append

def protoCleanupWasser(df):
    df2append = df.dropna()
    df2append = df2append[(df2append['ZählernummerAlt'] != 'Gerätenummer') &
                          (df2append['ZählerStandAlt'] != 'Stand in m³')&
                          (df2append['ZählernummerNeu'] != 'Gerätenummer') &
                          (df2append['ZählerStandNeu'] != 'Stand in m³')]
    df2append = df2append[(df2append['ZählernummerAlt'] != 'E') &
                          (df2append['ZählerStandAlt'] != 'J') &
                          (df2append['ZählernummerNeu'] != 'Nr.') &
                          (df2append['ZählerStandNeu'] != 'Mont. Pkt.')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Bowa')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Veha')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Fix')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Kermi')]
    return df2append

In [4]:
def collectProtocols(pathPattern='./absys_files/SenftenbergerRing*'):
    
    #Hier werden die Heizkostenverteilerdaten eingelesen
    protoColumns = {'Baulänge':str,
                    'Bauhöhe':str,
                    'Bautiefe':str,
                    'ZählernummerAlt':str,
                    'ZählerStandAlt':str,
                    'ZählernummerNeu':str}
    protokolle = glob.glob(pathPattern)
    #ToDo: Fehlerbehandlung wenn keine Dateien gefunden werden.
    protoDF = pd.DataFrame(columns=protoColumns.keys())
    for protokol in protokolle:
        protoWB = load_workbook(protokol)
        #Es wird vorrausgesetzt, dass das Worksheet "Dokumentation" heißt.
        try:
            protoWS = protoWB['Dokumentation']
        except:
            print("In der Datei "+protokol+" wurde kein Worksheet 'Dokumentation' gefunden!")
            break
        df = pd.read_excel(protokol, 
                           sheetname=0, 
                           names=protoColumns.keys(), 
                           dtype=protoColumns,  
                           parse_cols="E:G,L,R,V")
        df2appendHeizung = protoCleanupHeizung(df)
        pd.concat([protoDF, df2appendHeizung])
        #set_trace()
    #Hier werden die Heizkostenverteilerdaten eingelesen
    protoColumns = {'ZählernummerAlt':str,
                    'ZählerStandAlt':str,
                    'ZählernummerNeu':str,
                    'ZählerStandNeu':str,}
    protokolle = glob.glob(pathPattern)
    #ToDo: Fehlerbehandlung wenn keine Dateien gefunden werden.
    protoDF = pd.DataFrame(columns=protoColumns.keys())
    for protokol in protokolle:
        protoWB = load_workbook(protokol)
        #Es wird vorrausgesetzt, dass das Worksheet "Dokumentation" heißt.
        try:
            protoWS = protoWB['Dokumentation']
        except:
            print("In der Datei "+protokol+" wurde kein Worksheet 'Dokumentation' gefunden!")
            break
        df = pd.read_excel(protokol, 
                           sheetname=0, 
                           names=protoColumns.keys(), 
                           dtype=protoColumns,  
                           parse_cols="F,K,L,W")
        df2appendWasser = protoCleanupWasser(df)
        pd.concat([protoDF, df2appendWasser])
        #set_trace()
    protoDF.reset_index(drop=True, inplace=True)
    return protoDF

In [5]:
#Nimmt an, dass das Excel-sheet IMMER 'Zählerliste' heißt. Tippfehler gehen schief!!!
df = pd.read_excel(Auftrag, sheetname='Zählerliste', header=9)
protoDF = collectProtocols()
#Es wird vorrausgesetzt, das die Zaehlernummern immer in der 5.(E) Spalte stehen.
for Zählernummer in df['alte\nZähler-Nr']:
    #Fehlerbehandlung, wenn Feldinhalt keine Zählernummer
    try:
        proto = findZählernummer(Zählernummer, protoDF)
        if (len(proto['ZählernummerNeu'].values) > 0):
            #print('Zählernummer '+str(Zählernummer)+' gefunden')
            df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'neue\nZähler-Nr'] = proto['ZählernummerNeu'].values[0]
            #ToDo: Fehlerbehandlung wenn ein gültiger Zählerstand vorhanden.
            df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'Stand\nAusbau'] = proto['ZählerStandAlt'].values[0]
        else:
            print('Zählernummer '+str(Zählernummer)+' nicht gefunden')
    except:
        print('Bei der Bearbeitung von Zählernummer '+str(Zählernummer)+' ist ein Fehler aufgetreten')

Zählernummer 80552443 nicht gefunden
Zählernummer 80557588 nicht gefunden
Zählernummer 80552457 nicht gefunden
Zählernummer 80552454 nicht gefunden
Zählernummer 80557571 nicht gefunden
Zählernummer 80552456 nicht gefunden
Zählernummer 80556533 nicht gefunden
Zählernummer 80556532 nicht gefunden
Zählernummer 80552444 nicht gefunden
Zählernummer 44012657 nicht gefunden
Zählernummer 44012674 nicht gefunden
Zählernummer 44005513 nicht gefunden
Zählernummer 44005514 nicht gefunden
Zählernummer 84258799 nicht gefunden
Zählernummer 84265211 nicht gefunden
Zählernummer 84258756 nicht gefunden
Zählernummer 84258743 nicht gefunden
Zählernummer 84258737 nicht gefunden
Zählernummer 84651005 nicht gefunden
Zählernummer 84651001 nicht gefunden
Zählernummer 84258748 nicht gefunden
Zählernummer 84258739 nicht gefunden
Zählernummer 44012677 nicht gefunden
Zählernummer 44012658 nicht gefunden
Zählernummer 44005518 nicht gefunden
Zählernummer 44005515 nicht gefunden
Zählernummer 80416438 nicht gefunden
Z

Zählernummer 44005567 nicht gefunden
Zählernummer 80416485 nicht gefunden
Zählernummer 80416486 nicht gefunden
Zählernummer 80416488 nicht gefunden
Zählernummer 80416505 nicht gefunden
Zählernummer 80416494 nicht gefunden
Zählernummer 80416495 nicht gefunden
Zählernummer 44012666 nicht gefunden
Zählernummer 44005572 nicht gefunden
Zählernummer 80416506 nicht gefunden
Zählernummer 21924815 nicht gefunden
Zählernummer 80416500 nicht gefunden
Zählernummer 80416499 nicht gefunden
Zählernummer 80416502 nicht gefunden
Zählernummer 80416497 nicht gefunden
Zählernummer 43976276 nicht gefunden
Zählernummer 22975250 nicht gefunden
Zählernummer 80416519 nicht gefunden
Zählernummer 80416483 nicht gefunden
Zählernummer 80416482 nicht gefunden
Zählernummer 80416481 nicht gefunden
Zählernummer 80416467 nicht gefunden
Zählernummer 80416518 nicht gefunden
Zählernummer 43976254 nicht gefunden
Zählernummer 23476434 nicht gefunden
Zählernummer 80491129 nicht gefunden
Zählernummer 80491159 nicht gefunden
Z

Zählernummer 43976252 nicht gefunden
Zählernummer 23476430 nicht gefunden
Zählernummer 80490941 nicht gefunden
Zählernummer 80490943 nicht gefunden
Zählernummer 80490944 nicht gefunden
Zählernummer 80490932 nicht gefunden
Zählernummer 80490929 nicht gefunden
Zählernummer 43976251 nicht gefunden
Zählernummer 23476440 nicht gefunden
Zählernummer 80350645 nicht gefunden
Zählernummer 80286744 nicht gefunden
Zählernummer 80287047 nicht gefunden
Zählernummer 80491165 nicht gefunden
Zählernummer 43976273 nicht gefunden
Zählernummer 23476432 nicht gefunden
Zählernummer 80579494 nicht gefunden
Zählernummer 80566075 nicht gefunden
Zählernummer 80566076 nicht gefunden
Zählernummer 80579542 nicht gefunden
Zählernummer 43976249 nicht gefunden
Zählernummer 22975248 nicht gefunden
Zählernummer 80491322 nicht gefunden
Zählernummer 80491249 nicht gefunden
Zählernummer 80491318 nicht gefunden
Zählernummer 24028705 nicht gefunden
Zählernummer 44012669 nicht gefunden
Zählernummer 44005569 nicht gefunden
Z

Zählernummer 43978027 nicht gefunden
Zählernummer 43978021 nicht gefunden
Zählernummer 23977373 nicht gefunden
Zählernummer 23977380 nicht gefunden
Zählernummer 80425754 nicht gefunden
Zählernummer 80414582 nicht gefunden
Zählernummer 80425772 nicht gefunden
Zählernummer 80414578 nicht gefunden
Zählernummer 80425753 nicht gefunden
Zählernummer 80425755 nicht gefunden
Zählernummer 80313961 nicht gefunden
Zählernummer 80313962 nicht gefunden
Zählernummer 43978019 nicht gefunden
Zählernummer 43978013 nicht gefunden
Zählernummer 23977240 nicht gefunden
Zählernummer 23977377 nicht gefunden
Zählernummer 80425714 nicht gefunden
Zählernummer 80425791 nicht gefunden
Zählernummer 80425805 nicht gefunden
Zählernummer 80425716 nicht gefunden
Zählernummer 80313936 nicht gefunden
Zählernummer 80313935 nicht gefunden
Zählernummer 80425728 nicht gefunden
Zählernummer 80425733 nicht gefunden
Zählernummer 43978039 nicht gefunden
Zählernummer 43978026 nicht gefunden
Zählernummer 23977352 nicht gefunden
Z

Zählernummer 23456957 nicht gefunden
Zählernummer 23455610 nicht gefunden
Zählernummer 23456958 nicht gefunden
Zählernummer 23456900 nicht gefunden
Zählernummer 23456948 nicht gefunden
Zählernummer 43978023 nicht gefunden
Zählernummer 23977357 nicht gefunden
Zählernummer 80555860 nicht gefunden
Zählernummer 80556887 nicht gefunden
Zählernummer 80555840 nicht gefunden
Zählernummer 80555846 nicht gefunden
Zählernummer 80555838 nicht gefunden
Zählernummer 43978033 nicht gefunden
Zählernummer 23977356 nicht gefunden
Zählernummer 80558783 nicht gefunden
Zählernummer 80558800 nicht gefunden
Zählernummer 80558716 nicht gefunden
Zählernummer 80558777 nicht gefunden
Zählernummer 80558722 nicht gefunden
Zählernummer 43978028 nicht gefunden
Zählernummer 23977348 nicht gefunden
Zählernummer 80556376 nicht gefunden
Zählernummer 80558796 nicht gefunden
Zählernummer 80558776 nicht gefunden
Zählernummer 80556378 nicht gefunden
Zählernummer 80556347 nicht gefunden
Zählernummer 43978032 nicht gefunden
Z

Zählernummer 80416392 nicht gefunden
Zählernummer 80416394 nicht gefunden
Zählernummer 80416437 nicht gefunden
Zählernummer 80416423 nicht gefunden
Zählernummer 43685592 nicht gefunden
Zählernummer 43802502 nicht gefunden
Zählernummer 80417013 nicht gefunden
Zählernummer 80417012 nicht gefunden
Zählernummer 80416399 nicht gefunden
Zählernummer 80416398 nicht gefunden
Zählernummer 80416400 nicht gefunden
Zählernummer 43685593 nicht gefunden
Zählernummer 43802508 nicht gefunden
Zählernummer 80416403 nicht gefunden
Zählernummer 80416404 nicht gefunden
Zählernummer 80416407 nicht gefunden
Zählernummer 80416409 nicht gefunden
Zählernummer 80416405 nicht gefunden
Zählernummer 43685590 nicht gefunden
Zählernummer 43802507 nicht gefunden
Zählernummer 80416457 nicht gefunden
Zählernummer 80416458 nicht gefunden
Zählernummer 80416471 nicht gefunden
Zählernummer 80416431 nicht gefunden
Zählernummer 80416451 nicht gefunden
Zählernummer 43685584 nicht gefunden
Zählernummer 43802501 nicht gefunden
Z

Zählernummer 80286987 nicht gefunden
Zählernummer 40609097 nicht gefunden
Zählernummer 39903942 nicht gefunden
Zählernummer 43802511 nicht gefunden
Zählernummer 43802517 nicht gefunden
Zählernummer 80286753 nicht gefunden
Zählernummer 80286757 nicht gefunden
Zählernummer 80286761 nicht gefunden
Zählernummer 80286746 nicht gefunden
Zählernummer 80286766 nicht gefunden
Zählernummer 23537488 nicht gefunden
Zählernummer 40558046 nicht gefunden
Zählernummer 39903928 nicht gefunden
Zählernummer 43802518 nicht gefunden
Zählernummer 43802512 nicht gefunden
Zählernummer 80361300 nicht gefunden
Zählernummer 80361303 nicht gefunden
Zählernummer 80361304 nicht gefunden
Zählernummer 80363405 nicht gefunden
Zählernummer 80363403 nicht gefunden
Zählernummer 23271282 nicht gefunden
Zählernummer 40395379 nicht gefunden
Zählernummer 40558035 nicht gefunden
Zählernummer 43802513 nicht gefunden
Zählernummer 43802519 nicht gefunden
Zählernummer 80361271 nicht gefunden
Zählernummer 80363347 nicht gefunden
Z

Zählernummer 40004335 nicht gefunden
Zählernummer 43802542 nicht gefunden
Zählernummer 80313932 nicht gefunden
Zählernummer 80314097 nicht gefunden
Zählernummer 80314098 nicht gefunden
Zählernummer 80314092 nicht gefunden
Zählernummer 80314100 nicht gefunden
Zählernummer 23556974 nicht gefunden
Zählernummer 40321608 nicht gefunden
Zählernummer 43802543 nicht gefunden
Zählernummer 80314063 nicht gefunden
Zählernummer 80314071 nicht gefunden
Zählernummer 80314055 nicht gefunden
Zählernummer 80314075 nicht gefunden
Zählernummer 80314064 nicht gefunden
Zählernummer 23851407 nicht gefunden
Zählernummer 40004329 nicht gefunden
Zählernummer 43802537 nicht gefunden
Zählernummer 80314032 nicht gefunden
Zählernummer 80314133 nicht gefunden
Zählernummer 80314118 nicht gefunden
Zählernummer 80314080 nicht gefunden
Zählernummer 80314108 nicht gefunden
Zählernummer 23274239 nicht gefunden
Zählernummer 40350525 nicht gefunden
Zählernummer 43802538 nicht gefunden
Zählernummer 80314043 nicht gefunden
Z

In [6]:
protoDF

,ZählernummerAlt,ZählerStandAlt,ZählernummerNeu,ZählerStandNeu
